In [ ]:
SELECT MONTH(START_DATE) as MONTH, CAR_ID, COUNT(CAR_ID) as RECORDS
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY
WHERE YEAR(START_DATE) = 2022 and MONTH(START_DATE) between 8 and 10
and CAR_ID IN (
    SELECT CAR_ID
    FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY 
    WHERE YEAR(START_DATE) = 2022 and MONTH(START_DATE) between 8 and 10
    GROUP BY CAR_ID 
    HAVING count(*) >= 5
)
GROUP BY MONTH(START_DATE), CAR_ID
ORDER BY MONTH ASC, CAR_ID DESC

In [ ]:
SELECT CAR_ID,
    CASE 
        WHEN EXISTS (# CAR_Id 중에 하나가 조건에 만족 한다면
            SELECT 1
            FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY as c2
            WHERE '2022-10-16' between START_DATE and END_DATE and c1.CAR_ID = c2.CAR_ID # 이조건이 있으면 외부 쿼리에서 CAR_ID가 1001일때 내부 쿼리도 그 값에 대한 것만 조건을 검사하기 때문에 나누어서  검사함 
        )
        THEN '대여중'
        ELSE '대여 가능'
    END AS AVAILABILITY
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY as c1
GROUP BY CAR_ID
ORDER BY CAR_ID DESC

다음의 쿼리는 내부 쿼리와 외부 쿼리로 나눌 수 있다.
외부 쿼리는 AVAILABILITY 컬럼에 대해 '대여중' 또는 '대여가능' 중 하나의 값을 반환하게 된다.
이 때 내부 쿼리는 외부 쿼리의 CAR_ID를 참조하며, 이를 상관 서브쿼리 (correlated subquery) 라고 한다.
만약 상관 서브쿼리가 아니었다면, 서브쿼리가 하나라도 참이면 항상 '대여중'만 반환됐을 것이다.
하지만 현재는 내부 쿼리가 외부 쿼리의 CAR_ID 값을 받아와서,
외부에서 보고 있는 차량(CAR_ID = c1)에 대해 내부 쿼리에서 같은 차량(CAR_ID = c1)의 대여 여부를 개별적으로 판단하게 되는 것이다.

In [ ]:
# 외부쿼리
SELECT CAR_ID,
    CASE 
        WHEN EXISTS ()
        THEN '대여중'
        ELSE '대여 가능'
    END AS AVAILABILITY
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY as c1
GROUP BY CAR_ID
ORDER BY CAR_ID DESC

# 내부쿼리
SELECT 1
FROM CAR_RENTAL_COMPANY_RENTAL_HISTORY as c2
WHERE '2022-10-16' between START_DATE and END_DATE and c1.CAR_ID = c2.CAR_ID
        

---

In [ ]:
SELECT FOOD_TYPE, REST_ID, REST_NAME, FAVORITES
FROM REST_INFO as r1
WHERE FAVORITES = (
    SELECT MAX(FAVORITES)
    FROM REST_INFO as r2
    WHERE r1.FOOD_TYPE = r2.FOOD_TYPE
    GROUP BY FOOD_TYPE
)
ORDER BY FOOD_TYPE DESC

In [ ]:
SELECT r1.FOOD_TYPE, r1.REST_ID, r1.REST_NAME, r1.FAVORITES
FROM REST_INFO as r1
JOIN (
    SELECT FOOD_TYPE, MAX(FAVORITES) as MAX_FAV
    FROM REST_INFO
    GROUP BY FOOD_TYPE
) as r2
ON r1.FOOD_TYPE = r2.FOOD_TYPE and r1.FAVORITES = r2.MAX_FAV
ORDER BY r1.FOOD_TYPE DESC

In [ ]:
SELECT MCDP_CD as 진료과코드, COUNT(*) as 5월예약건수
FROM APPOINTMENT
WHERE YEAR(APNT_YMD) = 2022 and MONTH(APNT_YMD) = 5
GROUP BY MCDP_CD
ORDER BY 5월예약건수, MCDP_CD

In [ ]:
SELECT INGREDIENT_TYPE, SUM(TOTAL_ORDER) as TOTAL_ORDER
FROM FIRST_HALF as f
JOIN ICECREAM_INFO as I
ON f.FLAVOR = I.FLAVOR
GROUP BY INGREDIENT_TYPE
ORDER BY TOTAL_ORDER

In [ ]:
SELECT CAR_TYPE, COUNT(*) as CARS
FROM CAR_RENTAL_COMPANY_CAR as c1
WHERE OPTIONS LIKE '%가죽시트%'
    OR OPTIONS LIKE '%통풍시트%'
    OR OPTIONS LIKE '%열선시트%'
GROUP BY CAR_TYPE 
ORDER BY CAR_TYPE ASC